# **Emoji sentiment analysis model building code.**

**Importing the required libraries**

In [3]:
!pip install emoji
import emoji
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=8961124d9ca636fae77dfbc195fda1d13a7f65573e229fe309566956edfe1795
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


**Connecting Google drive**

**Not necessary if run on a local machine instead of google colab.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Check for the file path if run in local machine**

In [4]:
data = pd.read_csv('/content/drive/MyDrive/test/emoji_sence - Copy.csv', header = None)
data.head()

,0,1
0,grinning_face,4
1,beaming_face_with_smiling_eyes,4
2,face_with_tears_of_joy,4
3,rolling_on_the_floor_laughing,3
4,grinning_face_with_big_eyes,3


In [5]:
#data[0] = data[0].apply((lambda x: re.sub('_',' ',x)))
data[0] = data[0].apply((lambda x: re.sub('-','_',x)))

In [6]:
print(data[ data[1] >= 3].size)
print(data[ data[1] <= 1].size)
print(data[ data[1] == 2].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data[0].values)
X = tokenizer.texts_to_sequences(data[0].values)
X = pad_sequences(X)

46
42
16


In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 128)            256000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 5, 128)           0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 5)                 985       
                                                                 
Total params: 511,785
Trainable params: 511,785
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y = pd.get_dummies(data[1]).values
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X.shape,Y.shape)
#print(X_test.shape,Y_test.shape)

(52, 5) (52, 5)


In [ ]:
batch_size = 32
model.fit(X, Y, epochs = 7, batch_size=batch_size, verbose = 3)

Epoch 1/7
Epoch 2/7
Epoch 3/7
Epoch 4/7
Epoch 5/7
Epoch 6/7
Epoch 7/7


In [ ]:
validation_size = 15

X_validate = X[-validation_size:]
Y_validate = Y[-validation_size:]
X_test = X[:-validation_size]
Y_test = Y[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

2/2 - 0s - loss: 1.5268 - accuracy: 0.4054 - 359ms/epoch - 180ms/step
score: 1.53
acc: 0.41


In [ ]:
#pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
#for x in range(len(X_validate)):
    
#    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 0)
#  
 #   if np.argmax(result) == np.argmax(Y_validate[x]):
  #      if np.argmax(Y_validate[x]) <= 1:
   #         neg_correct += 1
    #    elif np.argmax(Y_validate[x]) >= 3:
     #       pos_correct += 1
     #  
    #if np.argmax(Y_validate[x]) <= 1:
    #    neg_cnt += 1
    #elif np.argmax(Y_validate[x]) >= 3:
    #    pos_cnt += 1



#print("pos_acc", pos_correct/pos_cnt*100, "%")
#print("neg_acc", neg_correct/neg_cnt*100, "%")

In [ ]:
string = input()
string = emoji.demojize(string)

#string = re.sub('_',' ',string)
string = re.sub('-','_',string)
string = string[1:-1]

string = np.array([string])

string = tokenizer.texts_to_sequences(string)
#padding the text to have exactly the same shape as `embedding_2` input
string = pad_sequences(string, maxlen=5, dtype='int32', value=0)
#print(twt)
sentiment = model.predict(string,batch_size=1)
print(np.argmax(sentiment))
if(np.argmax(sentiment) <= 1):
    print("negative")
elif(np.argmax(sentiment) == 2):
    print("neutral")
elif (np.argmax(sentiment) >= 3):
    print("positive")

😪
1
negative


**Check for the file path if run in local machine**

In [ ]:
fname = '/content/drive/MyDrive/test/finalized_emoji_model.sav'
pickle.dump(model, open(fname, 'wb'))
tokenname = '/content/drive/MyDrive/test/emoji_model_token.sav'
pickle.dump(tokenizer, open(tokenname, 'wb'))

INFO:tensorflow:Assets written to: ram://5ef7f875-5340-4f10-972e-41e0951692b5/assets


INFO:tensorflow:Assets written to: ram://5ef7f875-5340-4f10-972e-41e0951692b5/assets
